In [ ]:
import pandas as pd
import glob
from sklearn.preprocessing import MinMaxScaler

: 

## Ejercicio 2: Análisis de Funciones de Activación y Loss en Predicción de Series Temporales

Utilizarás el dataset público de calidad del aire de Beijing para crear un modelo de predicción de niveles de contaminación. Analizarás el impacto de diferentes funciones de activación y pérdida:

1. Preprocesamiento de datos:
   - Carga el dataset de calidad del aire de Beijing
   - Maneja valores faltantes
   - Normaliza las características
   - Crea secuencias temporales con ventana deslizante

In [36]:
# Lista de archivos CSV en la carpeta (ajusta el path si es necesario)
archivos = glob.glob("C:/Users/albaro/Documents/DataSets/beijingPollution/cities/*.csv")  # Puedes cambiar esto por una lista de nombres de archivos

# Cargar y concatenar todos los datasets
df = pd.concat([pd.read_csv(archivo) for archivo in archivos], ignore_index=True)

# Guardar el dataset combinado
df.to_csv("dataset_completo.csv", index=False)


In [37]:
# -------------------------------- QUITAR NULOS --------------------------------

# manejar los nulos con la mediana
# df.fillna(df.median(numeric_only=True), inplace=True)

# ffill para coger los anterires datos y rellenarlos por los null
df.ffill(inplace=True)

# Comprobar nulos
print(df.isnull().sum())

No         0
year       0
month      0
day        0
hour       0
PM2.5      0
PM10       0
SO2        0
NO2        0
CO         0
O3         0
TEMP       0
PRES       0
DEWP       0
RAIN       0
wd         0
WSPM       0
station    0
dtype: int64


In [ ]:
# -------------------------------- NORMALIZAR --------------------------------
# Definir columnas a normalizar (todas menos "wd", "station" y "year")
columnas_a_normalizar = df.drop(columns=["No","wd", "station", "year","month","day"]).columns

# Aplicar Min-Max Scaling
scaler = MinMaxScaler()
df[columnas_a_normalizar] = scaler.fit_transform(df[columnas_a_normalizar])

In [39]:
df.head()

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,0.000000,2013,3,1,0.000000,0.002006,0.002006,0.007433,0.020671,0.020202,0.071710,0.312195,0.672185,0.339310,0.0,NNW,0.333333,Aotizhongxin
1,0.000029,2013,3,1,0.043478,0.006018,0.006018,0.007433,0.020671,0.020202,0.071710,0.305691,0.675497,0.347586,0.0,N,0.356061,Aotizhongxin
2,0.000057,2013,3,1,0.086957,0.005015,0.005015,0.009434,0.031053,0.020202,0.067974,0.305691,0.680464,0.347586,0.0,NNW,0.424242,Aotizhongxin
3,0.000086,2013,3,1,0.130435,0.004012,0.004012,0.021441,0.034514,0.020202,0.067040,0.300813,0.697020,0.331034,0.0,NW,0.234848,Aotizhongxin
4,0.000114,2013,3,1,0.173913,0.001003,0.001003,0.023442,0.037974,0.020202,0.067040,0.291057,0.708609,0.329655,0.0,N,0.151515,Aotizhongxin


In [ ]:
# -------------------------------- SECUENCIA TEMPORAL --------------------------------

# Supongamos que df es tu DataFrame original

# Definir las columnas a utilizar (puedes incluir 'year', 'month', 'day', 'hour' si deseas)
columnas_a_utilizar = ["year", "month", "day", "hour", "PM10", "SO2", "NO2", "CO", "O3", "TEMP", "PRES", "DEWP", "RAIN", "WSPM"]

# Crear las características (X) y las etiquetas (y)
X = df[columnas_a_utilizar].iloc[:-1]  # Usamos todos los datos menos el último (X)
y = df["PM2.5"].iloc[1:]  # Usamos los datos de PM2.5 desplazados un paso para predecir el siguiente valor

# Ver las primeras filas para comprobar que las características y etiquetas se han creado correctamente
print(X.head())
print(y.head())

   year  month  day      hour      PM10       SO2       NO2        CO  \
0  2013      3    1  0.000000  0.002006  0.007433  0.020671  0.020202   
1  2013      3    1  0.043478  0.006018  0.007433  0.020671  0.020202   
2  2013      3    1  0.086957  0.005015  0.009434  0.031053  0.020202   
3  2013      3    1  0.130435  0.004012  0.021441  0.034514  0.020202   
4  2013      3    1  0.173913  0.001003  0.023442  0.037974  0.020202   

         O3      TEMP      PRES      DEWP  RAIN      WSPM  
0  0.071710  0.312195  0.672185  0.339310   0.0  0.333333  
1  0.071710  0.305691  0.675497  0.347586   0.0  0.356061  
2  0.067974  0.305691  0.680464  0.347586   0.0  0.424242  
3  0.067040  0.300813  0.697020  0.331034   0.0  0.234848  
4  0.067040  0.291057  0.708609  0.329655   0.0  0.151515  
1    0.006018
2    0.005015
3    0.004012
4    0.001003
5    0.003009
Name: PM2.5, dtype: float64


2. Implementa una red LSTM con:
   - 2 capas LSTM
   - Capas densas intermedias
   - Capa de salida para predicción

3. Experimenta con las siguientes combinaciones:
   Funciones de activación en capas intermedias:
   - ReLU
   - LeakyReLU
   - SELU
   - Tanh
   
   Funciones de pérdida:
   - MSE
   - MAE
   - Huber Loss
   - Log-cosh

4. Analiza y visualiza:
   - Error de predicción (RMSE y MAE)
   - Estabilidad del entrenamiento
   - Capacidad de generalización
   - Comportamiento con outliers